In [ ]:
import pandas as pd
import random
import numpy as np

from bs4 import BeautifulSoup
import requests
import re
import json
import csv

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep

pd.set_option('display.max_columns', None)

## url películas

In [ ]:
with open ('AccionDrama.json', 'r') as fichero:
    busqueda = json.load(fichero)

In [ ]:
lista_url = []

In [ ]:
for pelicula in busqueda:
    lista_url.append(f'https://www.imdb.com/title/{pelicula[0]}/fullcredits/?ref_=tt_ql_1')

## id actores en imdb

In [ ]:
lista_peliculas_y_actores = []
lista_todos_id = []

In [ ]:
for url in lista_url:
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()

    # cookies
    try:
        driver.find_element('css selector', '#imdb-bmo-footer > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
    except:
        pass

    sleep(1)

    id_actores_por_peli = [url]
    lista_href = []

    for actor in range (2, 12):
        xpath_actor = driver.find_element('xpath', f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{actor}]/td[2]/a')
        href_actor = xpath_actor.get_attribute('href')
        print(f'Voy por href: {href_actor}')
        lista_href.append(href_actor)

    for href in lista_href:
        patron_id = r'/name/(\w+)/'
        if 'https' in href:
            resultado = re.search(patron_id, href)
            id = resultado.group(1)
            print(f'Voy por id: {id}')
            id_actores_por_peli.append(id)
            lista_todos_id.append(id)
    
    driver.close()        
    
    lista_peliculas_y_actores.append(id_actores_por_peli)

## llamada a la api de actores

In [ ]:
json_info_actores = []

In [ ]:
for id in lista_todos_id:

    url = f"https://moviesdatabase.p.rapidapi.com/actors/{id}"
    headers = {
        "X-RapidAPI-Key": "902358e2e8msh3d16b2ef47d6b52p1fc5dajsnb0a5bd959614",
        "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers)

    json_info_actores(response.json())

### actores awards


In [ ]:
actores_texto = []


In [ ]:
for id in lista_todos_id:
    lista_awards = [id]
    try:
        driver.get(f'https://m.imdb.com/name/{id}/awards/')
        driver.find_element('css selector', '#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW').click()
        driver.maximize_window()

        awards = driver.find_element('css selector', '#__next > main > div > section > div').text.split('\n')
        lista_awards.append(awards)
    except:
        pass

    actores_texto.append(lista_awards)

In [ ]:
actores_awards = []

In [ ]:
for dato in actores_texto:
    for linea in dato[1]:
        patron = re.compile(r'\((\d+)\)')
        if patron.search(linea):
            actores_awards.append(linea)

            
